In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

In [2]:
# 환경 변수 불러오기
api_key = os.environ.get('MY_API_KEY')
# client 설정
client = genai.Client(api_key=api_key)

In [3]:
# 미리 정의된 스키마, 시스템 프롬포트 불러오기 (파일 이름으로 자동 탐색)
def find_file(filename, search_path='.'):
    # 주어진 파일명을 search_path부터 재귀적으로 탐색하여 경로 반환
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# 상위 디렉토리부터 탐색 시작
# 현재 노트북의 상위 디렉토리부터 탐색 시작
# base_path = pathlib.Path.cwd().parent.parent
# 현재 디렉토리부터 탐색 시작
base_path = pathlib.Path.cwd()

quality_validator_schema_path = find_file("QualityValidator_Object.json", base_path)
quality_validator_system_prompt_path = find_file("QualityValidator_SystemPrompt.md", base_path)

if not quality_validator_schema_path or not quality_validator_system_prompt_path:
    raise FileNotFoundError("Quality Validator 프롬포트 파일을 찾을 수 없습니다")

with open(quality_validator_schema_path, 'r', encoding='utf-8') as f:
    quality_validator_schema = json.load(f)

with open(quality_validator_system_prompt_path, 'r', encoding='utf-8') as f:
    quality_validator_system_prompt = f.read()

In [7]:
# 가짜 입력값: target_content, source(lecture_material), guideline(user_profile), required_count(target_card_count)
target_content = {
    "flash_cards": [
        {
            "id": 1,
            "front_content": "삼성전자의 2025년 4분기 잠정 매출액은 93조 원, 영업이익은 20.3조 원을 기록하며 전분기 대비 성장했습니다. 그러나 이 실적 규모가 시장의 이전 전망치와 비교했을 때 어떤 평가를 받았으며, 그 주된 이유는 무엇입니까?",
            "back_content": "이전 전망 대비 감소한 규모로 평가받았습니다. 이는 DS와 디스플레이는 예상 수준이었으나, MX 및 VD/가전 사업부의 수익성이 경기 부진과 원가 상승으로 인해 전망보다 크게 부진했기 때문입니다.",
            "category_tag": "2025 4Q 실적 총평"
        },
        {
            "id": 2,
            "front_content": "2025년 4분기 실적 개선의 핵심 동력이었던 DS(Device Solutions) 부문의 영업이익 증가(전분기 대비 10조 원 증가)를 이끈 구체적인 제품군과 가격 관련 요인은 무엇입니까?",
            "back_content": "DRAM과 SSD의 수요 증가에 따른 가격 상승(ASP 상승) 효과와 더불어 HBM 물량 증가가 영업이익 개선의 주요 원인이었습니다.",
            "category_tag": "DS 사업부"
        },
        {
            "id": 3,
            "front_content": "[시나리오] 당신은 경영 전략 회의에 참석 중입니다. MX(모바일) 사업부의 실적이 예상보다 크게 부진한 원인에 대해 질문을 받았을 때, '메모리 시장 상황'과 연계하여 어떻게 답변하시겠습니까?",
            "back_content": "메모리 반도체 가격의 상승으로 인해 스마트폰 제조를 위한 원가 구조가 악화된 것이 MX 사업부 수익성 부진의 주요 원인이라고 답변해야 합니다.",
            "category_tag": "MX 사업부"
        },
        {
            "id": 4,
            "front_content": "VD(영상디스플레이) 및 가전 사업부가 2025년 4분기에 큰 폭의 적자를 기록하게 만든 3가지 복합적인 외부 환경 리스크는 무엇입니까?",
            "back_content": "1) 내구재 소비 부진, 2) 시장 내 경쟁 심화, 3) 원재료 가격 상승입니다.",
            "category_tag": "CE/가전 사업부"
        },
        {
            "id": 5,
            "front_content": "2025년 4분기 메모리 반도체 출하량(Shipment)을 비교했을 때, DRAM과 NAND는 각각 어떤 상반된 수량 변동을 보였습니까?",
            "back_content": "DRAM은 낮은 한 자릿수(Low-single digit) 성장을 기록한 반면, NAND는 10% 감소하며 상반된 흐름을 보였습니다.",
            "category_tag": "메모리 출하량"
        },
        {
            "id": 6,
            "front_content": "[미래 전망] 보고서에 따르면 HBM 관련 우려가 2026년에 완화될 것으로 예상되는 이유는 무엇이며, 차세대 메모리인 HBM4 시장에서 삼성전자가 목표로 하는 지위는 무엇입니까?",
            "back_content": "AI 투자 확대에 따른 메모리 수요 본격화가 우려를 완화할 것이며, HBM4 시장에서 의미 있는 수준의 시장 점유율을 확보하는 것이 목표입니다.",
            "category_tag": "HBM4 전망"
        },
        {
            "id": 7,
            "front_content": "삼성전자의 목표주가가 180,000원으로 상향 조정된 재무적 산출 근거(Valuation)는 무엇입니까? (BPS와 PBR 수치를 포함하여 설명하시오)",
            "back_content": "2026년 예상 BPS(주당순자산가치)인 78,295원에 PBR(주가순자산비율) 2.3배를 적용하여 산출되었습니다.",
            "category_tag": "기업 가치 평가"
        },
        {
            "id": 8,
            "front_content": "2026년 삼성전자의 예상 영업이익은 약 151조 원으로 2025년(약 44조 원) 대비 비약적인 상승이 전망됩니다. 이처럼 실적이 급증할 것으로 기대되는 가장 결정적인 시장 배경은 무엇입니까?",
            "back_content": "AI 투자 확대에 따른 메모리 수요의 본격화와 DRAM/NAND 제품의 지속적인 가격 상승세가 예상되기 때문입니다.",
            "category_tag": "2026 실적 전망"
        },
        {
            "id": 9,
            "front_content": "[구조적 분석] 메모리 반도체 가격의 상승이 삼성전자 전체 수익성에 미치는 '양면성'에 대해 DS 부문과 완제품(MX/VD) 부문의 관점에서 각각 설명하십시오.",
            "back_content": "DS 부문에게는 매출 및 이익 성장을 이끄는 직접적인 호재가 되지만, 완제품(MX/VD) 부문에게는 부품 구매 비용 상승에 따른 원가 부담 가중으로 수익성을 악화시키는 요인이 됩니다.",
            "category_tag": "사업부별 인과관계"
        },

    ]
}
source = "/Users/jhkim/Desktop/LectureTestGenerator/SamSung(260109).pdf"
guideline = {
    "learning_goal": {
        "focus_areas": [
            "삼성전자 실적",
            "삼성전자 전망"
        ],
        "target_depth": "Deep Understanding",
        "question_modality": "Conceptual"
    },
    "user_status": {
        "proficiency_level": "Intermediate",
        "weakness_focus": True
    },
    "interaction_style": {
        "language_preference": "Korean_with_Korean_Terms",
        "scenario_based": True
    },
    "feedback_preference": {
        "strictness": "Strict",
        "explanation_depth": "Detailed_with_Examples"
    },
    "scope_boundary": "Lecture_Material_Only"
}
required_count = 10


# 강의 자료 파일 검사
# lecture_material 파일 읽기
path = pathlib.Path(source)

# 파일 타입별 처리
if path.is_file(): # 실제 존재하는 파일인지 먼저 검사
    # 파일 경로 소문자로 전환 (일관성 높이기 위해)
    suffix = path.suffix.lower()
    # PDF인 경우 처리
    if suffix == '.pdf':
        lecture_content = client.files.upload(file=path)
    # 텍스트 파일인 경우 처리
    elif suffix in ['.txt', '.md']:
        lecture_content = path.read_text(encoding='utf-8')
    # 지원되지 않는 파일은 경우
    else:
        lecture_content = "강의 자료 없음"
else:
    # 파일이 없다면, 파일 경로가 아닌, 강의 텍스트가 입력되었다고 처리
    lecture_content = source

quality_validator_contents = [
    "target_content: " + json.dumps(target_content),
    lecture_content,
    "guideline: " + json.dumps(guideline),
    "required_count: " + str(required_count)
]

In [8]:
# Response 생성
response = client.models.generate_content(
    model = "gemini-3-flash-preview",
    contents = quality_validator_contents,
    config={
        "system_instruction": quality_validator_system_prompt,
        "response_mime_type": "application/json",
        "response_schema": quality_validator_schema
    })

In [9]:
print(response.text)

{"is_valid":false,"feedback_message":[{"id":1,"message":"수량 및 시나리오 위반: 요청된 카드 수는 10개이나 9개만 생성되었습니다. 1개를 추가 생성하고, 'scenario_based: true' 지침에 따라 질문을 상황 제시형(예: '당신은 IR 담당자입니다...')으로 재구성하세요."},{"id":2,"message":"용어 및 시나리오 위반: 'Korean_with_Korean_Terms' 지침에 따라 ASP를 '평균판매단가'로, DS를 '반도체 사업부' 등 한글 용어로 수정하거나 병기하십시오. 또한 시나리오 형식을 도입하세요."},{"id":5,"message":"Depth 및 시나리오 위반: 단순 수치(Low-single digit 등) 암기는 'Deep Understanding' 목표에 부합하지 않습니다. DRAM과 NAND의 출하량 추이가 향후 시장 공급 과잉이나 부족에 어떤 시사점을 주는지 시나리오를 통해 질문하세요."},{"id":7,"message":"Depth 위반: 단순히 BPS와 PBR 수치를 묻는 것은 'Deep Understanding' 수준이 낮습니다. 2026년 예상 실적을 바탕으로 한 밸류에이션 상향의 전략적 근거와 타당성을 분석하도록 질문을 심화하세요."}]}
